In [1]:
#Read in libraries
import pandas as pd
import swifter

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

In [2]:
#Set style for matplotlib
plt.style.use('ggplot')

In [3]:
#Set options for pandas
pd.options.display.max_columns =1_000
pd.set_option('display.max_rows', 1_000)
pd.set_option('display.float_format', '{:.2f}'.format)

#Surpress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [4]:
#Read in data

path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\04_Models'

#Read in listings data
listings = pd.read_csv(path + '/01_19_2020_Listings_Processed.csv',  index_col=0)

#Ordinal variables to convert to category
ordinals = ['review_scores_accuracy', 'review_scores_checkin','review_scores_cleanliness', 
            'review_scores_communication','review_scores_location','review_scores_rating',
            'review_scores_value']

#Convert 
listings[ordinals] = listings[ordinals].astype('category')

# Preview Data

In [5]:
print('Listings shape: ', listings.shape)
listings.head()

Listings shape:  (65767, 469)


,accommodates,availability_30,availability_365,bathrooms,bedrooms,beds,chlc,chlc_private_rooms,chlc_shared_rooms,cleaning_fee,extra_people,guests_included,host_identity_verified,host_is_superhost,host_response_rate,instant_bookable,is_location_exact,maximum_maximum_nights,number_of_reviews,number_of_reviews_ltm,price,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,security_deposit,total_amenities,24-hour,Accessible-height,Air,Amazon,BBQ,Baby,Babysitter,Balcony,Bath,Bathroom,Bathtub,Beach,Beachfront,Bed,Bedroom,Bidet,Body,Breakfast,Building,Buzzer/wireless,Cable,Carbon,Cat(s),Ceiling,Central,Changing,Children’s,Cleaning,Coffee,Convection,Cooking,Crib,DVD,Day,Disabled,Dishes,Dishwasher,Dog(s),Doorman,Double,Dryer,Echo,Electric,Elevator,Espresso,Essentials,Ethernet,Exercise,Extra,Family/kid,Fax,Fire,Fireplace,Firm,First,Fixed,Flat,Formal,Free,Full,Game,Garden,Gas,Ground,Gym,HBO,Hair,Hammock,Handheld,Hangers,Heat,Heated,Heating,High,High-resolution,Host,Hot,Indoor,Internet,Iron,Jetted,Keypad,Kitchen,Kitchenette,Lake,Laptop,Lock,Lockbox,Long,Luggage,Memory,Microwave,Mini,Mobile,Mountain,Mudroom,Murphy,Netflix,Other,Outdoor,Outlet,Oven,Pack,Paid,Patio,Pets,Pillow-top,Play/travel,Pocket,Pool,Printer,Private,Projector,Rain,Refrigerator,Roll-in,Room-darkening,Safety,Self,Shampoo,Shared,Shower,Single,Ski-in/Ski-out,Smart,Smoke,Smoking,Soaking,Sound,Stair,Stand,Standing,Step-free,Stove,Suitable,Sun,Table,Tennis,Terrace,Toilet,Walk-in,Warming,Washer,Waterfront,Well-lit,Wheelchair,Wide,Wifi,Window,Wine,access,accessible,aid,air,allowed,alone,and,area,around,backyard,balcony,bars,basics,bath,bathroom,bed,bedroom,before,blankets,books,card,chair,charger,check-in,checkout,clearance,comforts,computer,conditioning,connection,console,cooler,corner,court,covers,crib,detector,dining,dinnerware,door,doorway,drawer,dropoff,dryer,en.hosting_amenity_49,en.hosting_amenity_50,enter,entrance,entryway,equipment,essentials,events,extinguisher,fan,fireplace,floor,floors,foam,for,fridge,friendly,front,gates,grab,greets,grill,guards,guest,guests,gym,hallway,hallways,head,hoist,home,hot,intercom,kettle,kit,kitchen,lamps,level,linens,live,living,lock,loungers,machine,maker,mattress,missing:,monitor,monoxide,off,oven,paper,parking,path,pet(s),pillows,pit,player,pool,premises,profiling,property,rack,recommendations,room,screen,seating,shades,shower,silverware,soap,space,spot,staff,stairs,stays,steam,steps,street,suite,system,table,term,this,toilet,towel,toys,translation,tub,valet,view,water,wifi,with,workspace,you,None,email,facebook,google,government_id,identity_manual,jumio,kba,manual_offline,manual_online,offline_government_id,phone,photographer,reviews,selfie,sent_id,weibo,work_email,zhima_selfie,host_identity_verified.1,host_is_superhost.1,instant_bookable.1,is_location_exact.1,require_guest_phone_verification.1,require_guest_profile_picture.1,requires_license.1,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,city_Daly City,city_San Francisco,city_South San Francisco,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,market_San Francisco,neighbourhood_cleansed_Bayview,neighbourhood_cleansed_Bernal Heights,neighbourhood_cleansed_Castro/Upper Market,neighbourhood_cleansed_Chinatown,neighbourhood_cleansed_Crocker Amazon,neighbourhood_cleansed_Diamond Heights,neighbourhood_cleansed_Downtown/Civic Center,neighbourhood_cleansed_Excelsior,neighbourhood_cleansed_Financial District,neighbourhood_cleansed_Glen Park,neighbourhood_c

# Split Data into Training and Test Sets

In [6]:
#Prepare data for splitting into train and test sets
X= listings.drop(columns='price').values
y= listings['price'].values

#Check shapes
print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (65767, 468)
y shape:  (65767,)


In [7]:
#Import models
from sklearn.feature_selection import SelectFromModel, SelectPercentile
from sklearn.model_selection import train_test_split

#Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   random_state=42)

#Check shapes
print('Training data shape: {}'.format(X_train.shape))
print('Test data shape: {}'.format(X_test.shape))

Training data shape: (49325, 468)
Test data shape: (16442, 468)


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [8]:
#Import models
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor



# #Instantiate Selectore with RandomForestRegressor
# select = SelectFromModel(RandomForestRegressor(random_state=42, n_jobs=-1,verbose=2,
#                                               n_estimators=100), threshold='median')

# #Fit and transform training data
# select.fit(X_train, y_train)
# X_train = select.transform(X_train)

# Base Model Pipeline

From a list of regressors, select the best model to optimize based on performance of initial application on training data

In [9]:
#Import ...
#Scalers
from sklearn.preprocessing import StandardScaler

#Feature_selection models
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2

#Supervised learning techniques
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge, Lasso,SGDRegressor

#Pipeline
from sklearn.pipeline import Pipeline

#CV
from sklearn.model_selection import cross_validate

In [10]:
selector = SelectFromModel(RandomForestRegressor(  #Extract top 1/2 of features
                                    random_state=42, n_jobs=-1,verbose=2,
                                    n_estimators=100), threshold='median')

#Create pipeline that scales data and extracts features
preprocessor = Pipeline(steps = [('Normalizer', StandardScaler()),#Normalize training data
    ('Feature_Selector',selector)])

#Fit to data
preprocessor.fit(X_train, y_train)

#Transform training data
X_train = preprocessor.transform(X_train)

#View new training data shape after transformation
print('New training data shape:', X_train.shape)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100building tree 8 of 100

building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100building tree 14 of 100

building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   28.9s


building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.7min finished


New training data shape: (49325, 234)


In [13]:
#Create list of regression models to iterate over and test in pipeline
regressors = [RandomForestRegressor(), 
              GradientBoostingRegressor(), 
              BaggingRegressor(),
             MLPRegressor(),
             SGDRegressor()]

* Create list of different scoring methods to evaluate model
* add to cross validate
* Remove linear regression, figure out which models do well with sparse data(SVM)
* Review models

In [14]:
#Run base models to see which performs best on training data
for regressor in regressors:
    pipe = Pipeline(steps=[('Model', regressor)], verbose=False) #Fit regressor
    scores  = cross_validate(pipe,X_train, y_train,scoring=('r2','neg_mean_absolute_error','neg_mean_squared_error'), cv =5)
    print('=============================================')
    print(regressor)
    print('Mean R2 Score: ', np.mean(scores['test_r2']))
    print('Mean MSE Score: ', np.mean(scores['test_neg_mean_squared_error']))
    print('Mean MAE Score: ', np.mean(scores['test_neg_mean_absolute_error']))
    print('=============================================')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
Mean R2 Score:  0.9419276139322944
Mean MSE Score:  -351.02501129274515
Mean MAE Score:  -8.394128388759407
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=10

C:\Users\kishe\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\kishe\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\kishe\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\kishe\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)
Mean R2 Score:  0.9389120673326617
Mean MSE Score:  -369.2678663118007
Mean MAE Score:  -11.172730736989479
SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)
Mean


#MSE “Mean squared error” is perhaps the most popular metric used for regression problems. 
#It essentially finds the average squared error between the predicted and actual values.

#MAE Mean absolute error (or mean absolute deviation) is another metric which finds the average 
#absolute distance between the predicted and target values.MAE is known to be more robust to the outliers than MSE


RandomForestRegressor will be the model of choice. Let's see if we can reduce that average MSE.

# Tune Hyperparameters

## Random Search

In [15]:
print('Random Forest parameters: ', RandomForestRegressor().get_params)

Random Forest parameters:  <bound method BaseEstimator.get_params of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)>


In [16]:
#Set values for RandomForestRegressor hyperparameters
max_depth = [15,20, 25, 30]

max_features= ['auto', 'sqrt', 'log2']

n_estimators = [10,50,75,100]

bootstrap=[True, False]

min_samples_leaf = [1,3,5]

param_grid = {'max_depth' : max_depth,
            'max_features':max_features,
            'n_estimators':n_estimators,
            'bootstrap': bootstrap,
            'min_samples_leaf' : min_samples_leaf,
}

In [17]:
from sklearn.model_selection import RandomizedSearchCV
random_model = RandomizedSearchCV(RandomForestRegressor(random_state = 42, n_jobs=-1), param_distributions= param_grid, n_jobs=-1,
                           scoring = ('neg_mean_squared_error'),
                                verbose =1)

print('View random_model parameters: ', random_model.get_params)

View random_model parameters:  <bound method BaseEstimator.get_params of RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=-1, oob_sc

In [18]:
search = random_model.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.2min finished


In [19]:
print(search.score(X_train, y_train))

search.best_params_

-36.86161171092091


{'n_estimators': 100,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

## Gridsearch

In [20]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#Set values for RandomForestRegressor hyperparameters
max_depth = [25, 30, 35]

max_features= ['auto', 'sqrt', 'log2']

n_estimators = [80,100,120]

bootstrap=[True, False]

min_samples_leaf = [1,2,3]

param_grid = {'max_depth' : max_depth,
            'max_features':max_features,
            'n_estimators':n_estimators,
            'bootstrap': bootstrap,
            'min_samples_leaf' : min_samples_leaf,
}

In [ ]:
GridSearchCV(RandomForestRegressor(), X_train, y_train, n_jobs=)

# Deploy